# Análisis Exploratorio
## Tema 13: Identificación de menciones de entidades biomédicas en resúmenes de investigación

Integrantes:
- José Ricardo Méndez González, 21289
- Sara María Pérez Echeverría, 21371
- Emily Elvia Melissa Pérez Alarcón, 21385
- Adrian Fulladolsa Palma, 21592

### Librerías

### Importación de los datos